In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


class NDVILandCoverClassifier:
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        # Use RandomForest but with more conservative parameters
        self.model = RandomForestClassifier(
            n_estimators=100,           # Fewer trees to reduce overfitting
            max_depth=10,               # More controlled depth
            min_samples_split=10,       # Require more samples for splits
            min_samples_leaf=4,         # Require more samples in leaf nodes
            max_features='sqrt',        # Use sqrt(n_features) for each split decision
            bootstrap=True,
            oob_score=True,             # Out-of-bag scoring for monitoring
            class_weight='balanced',    # Balance class weights
            random_state=random_state,
            n_jobs=-1                   # Use all available cores
        )
        self.ndvi_columns = []
        
    def extract_ndvi_columns(self, df):
        """Extract NDVI time series columns"""
        ndvi_cols = [col for col in df.columns if col.endswith('_N')]
        ndvi_cols.sort()
        return ndvi_cols
    
    def clean_data(self, df, ndvi_cols):
        """Clean NDVI data"""
        df_clean = df.copy()
        
        print("Converting NDVI columns to numeric...")
        for col in ndvi_cols:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        
        return df_clean
    
    def preprocess_ndvi(self, df, ndvi_cols):
        """Basic preprocessing of NDVI values"""
        df_processed = df.copy()
        
        print("Processing NDVI data...")
        
        # Simple median imputation for missing values
        for col in ndvi_cols:
            median = df_processed[col].median()
            if np.isnan(median):
                median = 0.3  # Reasonable NDVI value
            df_processed[col].fillna(median, inplace=True)
        
        # Handle outliers with clipping
        for col in ndvi_cols:
            # NDVI should be between -1 and 1, but source data might have different scaling
            q1 = df_processed[col].quantile(0.01)  # 1st percentile
            q3 = df_processed[col].quantile(0.99)  # 99th percentile
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            df_processed[col] = df_processed[col].clip(lower, upper)
        
        # Rescale NDVI to standard -1 to 1 range if necessary
        max_abs_ndvi = max(abs(df_processed[ndvi_cols].min().min()), 
                          abs(df_processed[ndvi_cols].max().max()))
        
        # Only normalize if the values seem to be on a different scale
        if max_abs_ndvi > 3:  # If values are far outside normal NDVI range
            print("Rescaling NDVI values to standard range...")
            for col in ndvi_cols:
                df_processed[col] = df_processed[col] / max_abs_ndvi
        
        # Final clip to ensure -1 to 1 range
        for col in ndvi_cols:
            df_processed[col] = df_processed[col].clip(-1, 1)
            
        return df_processed
    
    def extract_features(self, df, ndvi_cols):
        """Extract features focused on land cover types"""
        print("Extracting features...")
        features_df = pd.DataFrame(index=df.index)
        
        # Convert to numpy array for faster processing
        ndvi_data = df[ndvi_cols].values
        
        # Basic statistical features
        features_df['ndvi_mean'] = np.mean(ndvi_data, axis=1)
        features_df['ndvi_std'] = np.std(ndvi_data, axis=1)
        features_df['ndvi_min'] = np.min(ndvi_data, axis=1)
        features_df['ndvi_max'] = np.max(ndvi_data, axis=1)
        features_df['ndvi_range'] = features_df['ndvi_max'] - features_df['ndvi_min']
        features_df['ndvi_median'] = np.median(ndvi_data, axis=1)
        
        # Vegetation presence counts at different thresholds
        # High NDVI values indicate dense vegetation (forest)
        features_df['ndvi_above_02_count'] = (ndvi_data > 0.2).sum(axis=1)
        features_df['ndvi_above_04_count'] = (ndvi_data > 0.4).sum(axis=1)
        features_df['ndvi_above_06_count'] = (ndvi_data > 0.6).sum(axis=1)
        
        # Low/negative NDVI values indicate water or bare surfaces
        features_df['ndvi_below_0_count'] = (ndvi_data < 0).sum(axis=1)
        features_df['ndvi_below_02_count'] = (ndvi_data < 0.2).sum(axis=1)
        
        # Quarter analysis for seasonal patterns
        n_points = len(ndvi_cols)
        quarter_size = n_points // 4
        
        for i in range(4):
            start_idx = i * quarter_size
            end_idx = (i + 1) * quarter_size if i < 3 else n_points
            quarter_data = ndvi_data[:, start_idx:end_idx]
            
            # Quarterly statistics
            features_df[f'ndvi_q{i+1}_mean'] = np.mean(quarter_data, axis=1)
            features_df[f'ndvi_q{i+1}_max'] = np.max(quarter_data, axis=1)
        
        # Seasonal change detection (simple but effective)
        for i in range(3):  # Compare adjacent quarters
            features_df[f'ndvi_q{i+1}_to_q{i+2}_diff'] = (
                features_df[f'ndvi_q{i+2}_mean'] - features_df[f'ndvi_q{i+1}_mean']
            )
        
        # Add some raw NDVI values (evenly spaced across time)
        indices = np.linspace(0, n_points-1, 5).astype(int)
        for i, idx in enumerate(indices):
            if idx < len(ndvi_cols):
                features_df[f'ndvi_t{i}'] = df[ndvi_cols[idx]]
        
        # Fill any remaining NaN values
        features_df = features_df.fillna(0)
        
        print(f"Generated {features_df.shape[1]} features")
        return features_df
    
    def preprocess_data(self, df, is_training=True):
        """Complete preprocessing pipeline"""
        print("Starting preprocessing...")
        
        # Extract NDVI columns
        self.ndvi_columns = self.extract_ndvi_columns(df)
        print(f"Found {len(self.ndvi_columns)} NDVI time points")
        
        # Clean and convert data
        df_clean = self.clean_data(df, self.ndvi_columns)
        
        # Preprocess NDVI values
        df_processed = self.preprocess_ndvi(df_clean, self.ndvi_columns)
        
        # Extract features
        features_df = self.extract_features(df_processed, self.ndvi_columns)
        
        # Add ID column if present
        if 'ID' in df.columns:
            features_df['ID'] = df['ID']
        
        return features_df, df_processed
    
    def fit(self, X_train, y_train):
        """Train the model with careful monitoring"""
        print("Training Random Forest model...")
        
        # Encode labels
        y_encoded = self.label_encoder.fit_transform(y_train)
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X_train)
        
        # Train model
        self.model.fit(X_scaled, y_encoded)
        
        # Report out-of-bag score (built-in cross-validation)
        print(f"Out-of-bag score: {self.model.oob_score_:.4f}")
        
        # Full evaluation on training data
        y_pred = self.model.predict(X_scaled)
        print("\nClassification Report (on training data):")
        print(classification_report(y_encoded, y_pred, target_names=self.label_encoder.classes_))
        
        # Class distribution analysis
        pred_class_counts = pd.Series(self.label_encoder.inverse_transform(y_pred)).value_counts()
        print("\nPredicted class distribution (training):")
        print(pred_class_counts)
        
        return self
    
    def predict(self, X_test):
        """Make predictions"""
        X_scaled = self.scaler.transform(X_test)
        y_pred_encoded = self.model.predict(X_scaled)
        y_pred = self.label_encoder.inverse_transform(y_pred_encoded)
        return y_pred
    
    def predict_proba(self, X_test):
        """Get prediction probabilities"""
        X_scaled = self.scaler.transform(X_test)
        return self.model.predict_proba(X_scaled)


def main():
    """Main training and prediction pipeline"""
    print("NDVI Land Cover Classification Pipeline")
    print("=" * 50)
    
    # Initialize classifier
    classifier = NDVILandCoverClassifier(random_state=42)
    
    try:
        # Load training data
        print("Loading training data...")
        train_df = pd.read_csv('hacktrain.csv')
        print(f"Training data shape: {train_df.shape}")
        print(f"Classes: {train_df['class'].value_counts()}")
        
        # Preprocess training data
        X_train_processed, train_df_clean = classifier.preprocess_data(train_df, is_training=True)
        
        # Prepare features and target
        feature_columns = [col for col in X_train_processed.columns if col not in ['ID', 'class']]
        X_train = X_train_processed[feature_columns]
        y_train = train_df['class']
        
        print(f"Training features shape: {X_train.shape}")
        
        # Train model
        classifier.fit(X_train, y_train)
        
        # Analyze feature importance
        analyze_feature_importance(classifier, feature_columns)
        
        # Load test data
        print("\nLoading test data...")
        test_df = pd.read_csv('hacktest.csv')
        print(f"Test data shape: {test_df.shape}")
        
        # Preprocess test data
        X_test_processed, _ = classifier.preprocess_data(test_df, is_training=False)
        X_test = X_test_processed[feature_columns]
        
        # Make predictions
        print("Making predictions...")
        predictions = classifier.predict(X_test)
        
        # Prepare submission
        submission_df = pd.DataFrame({
            'ID': test_df['ID'],
            'class': predictions
        })
        
        # Save submission
        submission_df.to_csv('submission.csv', index=False)
        print(f"\nSubmission saved to 'submission.csv'")
        print(f"Predictions distribution:")
        print(submission_df['class'].value_counts())
        
        # Display sample predictions
        print(f"\nSample predictions:")
        print(submission_df.head(10))
        
        return classifier, submission_df
        
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        print("Full error traceback:")
        traceback.print_exc()
        return None, None


def analyze_feature_importance(classifier, feature_names):
    """Analyze feature importance from the trained model"""
    if hasattr(classifier.model, 'feature_importances_'):
        # Get feature importances
        importances = classifier.model.feature_importances_
        
        # Create feature importance dataframe
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        print("\nTop 15 Most Important Features:")
        print(importance_df.head(15))
        
        return importance_df
    else:
        print("Feature importances not available")
        return None


if __name__ == "__main__":
    classifier, submission = main()
    
    if classifier is not None:
        print("\nPipeline completed successfully!")
        print("Check your directory for 'submission.csv' file.")
    else:
        print("\nPipeline failed. Please check the error messages above.")

NDVI Land Cover Classification Pipeline
Loading training data...
Training data shape: (8000, 30)
Classes: class
forest        6159
farm           841
impervious     669
grass          196
water          105
orchard         30
Name: count, dtype: int64
Starting preprocessing...
Found 27 NDVI time points
Converting NDVI columns to numeric...
Processing NDVI data...
Rescaling NDVI values to standard range...
Extracting features...
Generated 27 features
Training features shape: (8000, 27)
Training Random Forest model...
Out-of-bag score: 0.8769

Classification Report (on training data):
              precision    recall  f1-score   support

        farm       0.65      0.93      0.77       841
      forest       0.99      0.92      0.95      6159
       grass       0.71      0.97      0.82       196
  impervious       0.91      0.91      0.91       669
     orchard       0.56      1.00      0.71        30
       water       0.94      0.96      0.95       105

    accuracy                  